## Week 13 Medium - Line Notify的應用

In [1]:
import tejapi 
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

tejapi.ApiConfig.api_key = "yourkey"
tejapi.ApiConfig.ignoretz = True

### 資料撈取(TEJAPI)及整理

In [2]:
ticks = ['2330', '2303', '1101']

DailyPrice= tejapi.get('TWN/EWPRCD',coid=ticks,
                        opts={'columns':['coid', 'mdate', 'open_d', 'high_d','low_d', 'close_d']},
                        mdate={'gte':'2020-05-01','lte':'2021-05-25'}, paginate=True, )
DailyPrice= DailyPrice.set_index('mdate')

In [3]:
## 周線；月線；日變化

MovingAvg_5D = {}
MovingAvg_20D = {}
DailyRt = {}

for ticker in ticks:
    MovingAvg_5D[ticker] = DailyPrice[DailyPrice['coid']==ticker]['close_d'].rolling(5).mean()
    MovingAvg_20D[ticker] = DailyPrice[DailyPrice['coid']==ticker]['close_d'].rolling(20).mean()
    DailyRt[ticker] = DailyPrice[DailyPrice['coid']==ticker]['close_d'].pct_change()*100

In [4]:
MovingAvg_20D['2330']

mdate
2020-05-04       NaN
2020-05-05       NaN
2020-05-06       NaN
2020-05-07       NaN
2020-05-08       NaN
               ...  
2021-05-19    583.45
2021-05-20    582.20
2021-05-21    581.30
2021-05-24    579.60
2021-05-25    578.25
Name: close_d, Length: 260, dtype: float64

### 連接Line Notify

In [ ]:
token = "your token"

def LineNotify(params, token):
    
    headers = {
        "Authorization": "Bearer " + token,
        "Content-Type": "application/x-www-form-urlencoded"
        }
    r = requests.post("https://notify-api.line.me/api/notify", headers=headers, params=params)
    print(r.status_code)

In [ ]:
import requests
params = []
for ticker in ticks:
    
    if MovingAvg_20D[ticker][-1] > MovingAvg_20D[ticker][-2] and MovingAvg_5D[ticker][-1] > MovingAvg_5D[ticker][-2]:

        params.append('\n' + ticker + ':\n' 
                      + "五日均線往上為: " + str(round(MovingAvg_5D[ticker][-1],2)) + '\n' 
                      + "月均線往上為: " + str(round(MovingAvg_20D[ticker][-1],2)) + '\n' 
                      + "日報酬率為: " + str(round(DailyRt[ticker][-1],2)) + '%' + '\n' 
                      + "當前價格: " + str(round(DailyPrice[DailyPrice['coid']==ticker]['close_d'][-1])) + '\n')
    
    elif MovingAvg_20D[ticker][-1] > MovingAvg_20D[ticker][-2]:

        params.append('\n' + ticker + ':\n' 
                      + "月均線往上為: " + str(round(MovingAvg_20D[ticker][-1],2)) + '\n' 
                      + "日報酬率為: " + str(round(DailyRt[ticker][-1],2)) + '%' + '\n' 
                      + "當前價格: " + str(round(DailyPrice[DailyPrice['coid']==ticker]['close_d'][-1])) + '\n')
    
    elif MovingAvg_5D[ticker][-1] > MovingAvg_5D[ticker][-2]:

        params.append('\n' + ticker + ':\n' 
                      + "五日均線往上為: " + str(round(MovingAvg_5D[ticker][-1],2)) + '\n'  
                      + "日報酬率為: " + str(round(DailyRt[ticker][-1],2)) + '%' + '\n' 
                      + "當前價格: " + str(round(DailyPrice[DailyPrice['coid']==ticker]['close_d'][-1])) + '\n')  
    
params = {'message': params}
LineNotify(params, token)